##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 케라스 모델로 추정기 생성하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/keras_model_to_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">구글 코랩(Colab)에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">깃허브(GitHub) 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a></td>
</table>

> 경고: 추정기는 새 코드에 권장되지 않습니다. Estimator는 `v1.Session` 스타일 코드를 실행합니다. 이 코드는 올바르게 작성하기가 더 어렵고 특히 TF 2 코드와 결합될 때 예기치 않게 작동할 수 있습니다. 에스티메이터는 [호환성 보장](https://tensorflow.org/guide/versions) 이 적용되지만 보안 취약점 외에는 수정 사항이 제공되지 않습니다. 자세한 내용은 [마이그레이션 가이드](https://tensorflow.org/guide/migrate) 를 참조하세요.

## 개요

TensorFlow Estimator는 TensorFlow에서 지원되며 신규 및 기존 `tf.keras` 모델에서 생성할 수 있습니다. 이 자습서에는 해당 프로세스의 완전하고 최소한의 예가 포함되어 있습니다.

주의: 케라스 모델을 사용한다면, 추정량을 변환하지 않고 [`tf.distribute` strategies](https://tensorflow.org/guide/migrate/guide/distributed_training)과 함께 직접 사용할 수 있습니다. 따라서, `model_to_estimator`s는 더 이상 권장되지 않습니다.

## 설정

In [ ]:
import tensorflow as tf

import numpy as np
import tensorflow_datasets as tfds

### 간단한 케라스 모델 만들기

케라스에서는 여러 겹의 층을 쌓아 모델을 만들 수 있습니다. 일반적으로 모델은 층의 그래프로 구성됩니다. 이 중 가장 흔한 형태는 적층형 구조를 갖고 있는 [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) 모델입니다.

간단한 완전히 연결 네트워크(다층 퍼셉트론)를 만들어봅시다:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)
])

모델을 컴파일한 후, 모델 구조를 요약해 출력할 수 있습니다.

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam')
model.summary()

### 입력 함수 만들기

[데이터셋 API](../../guide/data.md)를 사용해 대규모 데이터셋을 다루거나 여러 장치에서 훈련할 수 있습니다.

텐서플로 추정기는 입력 파이프라인(input pipeline)이 언제 어떻게 생성되었는지 제어해야 합니다. 이를 위해서는 "입력 함수", 즉 `input_fn`이 필요합니다. 추정기는 이 함수를 별도의 매개변수 설정 없이 호출하게 됩니다. 이때 `input_fn`은 `tf.data.Dataset` 객체를 반환해야 합니다.

In [ ]:
def input_fn():
  split = tfds.Split.TRAIN
  dataset = tfds.load('iris', split=split, as_supervised=True)
  dataset = dataset.map(lambda features, labels: ({'dense_input':features}, labels))
  dataset = dataset.batch(32).repeat()
  return dataset

`input_fn`이 잘 구현되었는지 확인해봅니다.

In [ ]:
for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)

### tf.keras.model을 추정기로 변환하기

`tf.keras.model`은 `tf.keras.estimator.model_to_estimator` 함수를 이용해 `tf.estimator.Estimator` 객체로 변환함으로써 `tf.estimator` API를 통해 훈련할 수 있습니다.

In [ ]:
import tempfile
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model, model_dir=model_dir)

추정기를 훈련한 후 평가합니다.

In [ ]:
keras_estimator.train(input_fn=input_fn, steps=500)
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)
print('Eval result: {}'.format(eval_result))